In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pint import UnitRegistry

import scipy as sp
from scipy.optimize import fsolve

from fluid import Fluid

import json

with open('turbine_conditions.json') as f:
    conditions = json.load(f)



In [33]:
ureg = UnitRegistry()
co2 = Fluid(fluid='co2', ureg=ureg)

Q_ = ureg.Quantity
m_out = Q_(0.26,  ureg.kilogram / ureg.second)
t_out = 150 + 273.15
delta_p = Q_(2.1 * 1e6,  ureg.pascal)

def strip(quantity):
    # check if quantity is a pint quantity type
    if isinstance(quantity, Q_):
        # if it is, return the magnitude
        return quantity.magnitude
    else:
        # otherwise return the value
        return quantity


In [41]:
def find_mass_ratio(m_tot, h_tot, h1, h2):
    # confirm units and make dimensionless
    m_tot = strip(m_tot)
    h_tot = strip(h_tot)
    h1 = strip(h1)
    h2 = strip(h2)
    
    def enthalpy_balance(m1):
        m2 = m_tot - m1
        return h_tot * (m1 + m2) - (h1 * m1 + h2 * m2)
    
    
    m1_0 = 0.5 * m_tot
    naive_solve = fsolve(enthalpy_balance, m1_0)
    if naive_solve > 0:
        return naive_solve
    else:
        return ()

In [42]:
for condition in conditions:
    for flow_1 in range(1,7):
        
        enthalpy1 = co2.enthalpy(P = condition[f'P{flow_1}'], T = condition[f'T{flow_1}'])
        
        for flow_2 in range(flow_1 + 1,7):
            
            enthalpy2 = co2.enthalpy(P = condition[f'P{flow_2}'], T = condition[f'T{flow_2}'])
            
            for exhaust in range(1,7):
                if exhaust == flow_1 or exhaust == flow_2:
                    continue
                p_tot = Q_(condition[f'P{exhaust}'], ureg.pascal) + delta_p
                h_tot = co2.enthalpy(P = p_tot, T = t_out)
                
                #print(f"m_out={m_out}, h_tot={h_tot}, e1={enthalpy1}, e2={enthalpy2}")
                
                m1_range = find_mass_ratio(m_out, h_tot, enthalpy1, enthalpy2)
                if m1_range == ():
                    pass
                    #print(f"X {flow_1}; {flow_2} -> {exhaust}")
                else:
                    print(f"for m1 = {flow_1} m2 = {flow_2} exhaust = {exhaust} the range of m1 is {m1_range}")
    
    
    
    

/tmp/ipykernel_2079850/2870282830.py:19: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if m1_range == ():
/tmp/ipykernel_2079850/1115663762.py:14: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  naive_solve = fsolve(enthalpy_balance, m1_0)


for m1 = 1 m2 = 4 exhaust = 2 the range of m1 is [0.06984841]
for m1 = 1 m2 = 4 exhaust = 3 the range of m1 is [0.06984841]
for m1 = 1 m2 = 4 exhaust = 5 the range of m1 is [0.0409445]
for m1 = 1 m2 = 4 exhaust = 6 the range of m1 is [0.0409445]
for m1 = 1 m2 = 5 exhaust = 2 the range of m1 is [0.02811126]
for m1 = 1 m2 = 5 exhaust = 3 the range of m1 is [0.02811126]
for m1 = 1 m2 = 5 exhaust = 4 the range of m1 is [0.02811126]
for m1 = 2 m2 = 4 exhaust = 1 the range of m1 is [0.04188691]
for m1 = 2 m2 = 4 exhaust = 3 the range of m1 is [0.0714561]
for m1 = 2 m2 = 4 exhaust = 5 the range of m1 is [0.04188691]
for m1 = 2 m2 = 4 exhaust = 6 the range of m1 is [0.04188691]
for m1 = 2 m2 = 5 exhaust = 3 the range of m1 is [0.02890432]
for m1 = 2 m2 = 5 exhaust = 4 the range of m1 is [0.02890432]
for m1 = 3 m2 = 4 exhaust = 1 the range of m1 is [0.05710065]
for m1 = 3 m2 = 4 exhaust = 2 the range of m1 is [0.09740966]
for m1 = 3 m2 = 4 exhaust = 5 the range of m1 is [0.05710065]
for m1 = 3 